# Weird winds continued


Chris suggestes that a lack of vertical momentum mixing in the coarse resolution SAM could be causing a rapid degredation of the wind fields in the extra tropics when initialized with data from NG-Aqua.

First, let's open the 2D outputs:

In [ ]:
import xarray as xr

In [ ]:
runs = {
    'NGAqua': '/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc',
    'NoNN': '../../data/runs/khyp_1e16/OUT_2D/NG1_test_1.2Dbin_1.nc',
    'NN': '/Users/noah/workspace/research/uwnet/trained_models/5b/92d2bbe6060eb1f80f945e/SAM/OUT_2D/NG1_test_1.2Dbin_1.nc',
    'NN_noSOLIN': '/Users/noah/workspace/research/uwnet/trained_models/5b/92fc43e6060e6a470454ec/SAM/OUT_2D/NG1_test_1.2Dbin_1.nc'
}


time_range = slice(100, 103)


# open data
base =( xr.open_mfdataset(runs['NGAqua'])
       .sortby('time')
       .sel(time=time_range))

datasets = {}
for key, val in runs.items():
    print(f"Opening {key}")
    datasets[key] = xr.open_dataset(val).sortby('time').sel(time=base.time, method='nearest')

## VSFC

This is the plot I showed in the notebook, during our meeting. It shows the VSFC winds around y= 8000 km disappearing.

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,6), constrained_layout=True)

for ax, (key, val) in zip(axs.flat, datasets.items()):
    val.VSFC.mean('x').plot(x='time', ax=ax)
    ax.set_title(key)

## V850

Herea the 850 hPA y-winds. They show a substnatially different y structure.

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,6), constrained_layout=True)

for ax, (key, val) in zip(axs.flat, datasets.items()):
    val.V850.mean('x').plot(x='time', ax=ax)
    ax.set_title(key)

The meridional winds at V850 look substantially different than the surface winds.

## 3D winds

I plot the full 3D structure of the zonal mean winds at y= 8480 km here.

### Meridional winds

In [ ]:
csam_no_nn_3d = xr.open_mfdataset("../../data/runs/khyp_1e16/OUT_3D/*.nc", autoclose=True)
csam_nn_3d = xr.open_mfdataset("/Users/noah/workspace/research/uwnet/trained_models/5b/92fc43e6060e6a470454ec/SAM/OUT_3D/*.nc", autoclose=True)

ng_3d = xr.open_dataset("/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/stagger/3d/all.nc")
ng_3d_centered = xr.open_dataset("/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/all.nc")


def plot_v(v, a=2.0):
    v.sel(time=slice(100,103)).plot(y='z', vmax=a, vmin=-a, cmap='RdBu_r')


plot_v(csam_no_nn_3d.V.isel(y=53).mean('x'))
plt.title("coarse SAM (cSAM)")


plt.figure()
plot_v(ng_3d.V.isel(ys=53).mean('xc'))
plt.title("NGAqua")

We can see here are big differences in the winds structure throught the atmosphere. Most obviously, the sponge layer does not appear to be on, so there are large variations in the upper troposphere of the cSAM simulation. There is also a regular oscillation at the surface.

### Zonal winds

In [ ]:
plot_v(csam_no_nn_3d.U.isel(y=53).mean('x'), a=100)
plt.title("coarse SAM")


plt.figure()
plot_v(ng_3d.U.isel(yc=53).mean('xs'), a=100)
plt.title("NGAqua")


## Zonal mean meridional wind plots for first 3 days

### Coarse Resolution SAM (no NN)

In [ ]:
csam_no_nn_3d.V[::4].mean('x').plot(col='time', col_wrap=4)

### NGAqua

In [ ]:
ng_3d.V[::4].sel(time=slice(100, 103.2)).mean('xc').plot(col='time', col_wrap=4)

## Zonal mean plots

### Coarse Res SAM

In [ ]:
csam_no_nn_3d.U[::4].mean('x').plot(col='time', col_wrap=4)

### NG-Aqua

In [ ]:
ng_3d.U[::4].sel(time=slice(100, 103.2)).mean('xs').plot(col='time', col_wrap=4)

# Conclusions

1. I need to re-enable the gravity wave damper in the cSAM runs.
2. I probably need to impose some boundary layer friction somehow, to ensure that $ -f V_{sfc} \approx  \text{zonal drag} $. This would maintain positive meridional velocity in the region of the surface westerlies.